In [4]:
import os, requests, math, pandas as pd
from geopy import Nominatim
from shapely.geometry import Point

def geocode_city(city, country=None, radius_km=5):
    loc = Nominatim(user_agent="gap-index").geocode(f"{city}, {country or ''}")
    if not loc:
        raise ValueError("City not found")
    return {"lat": loc.latitude, "lon": loc.longitude, "radius_m": radius_km*1000}

city_geom = geocode_city("Santiago", "Chile", radius_km=5)


In [11]:
import overpy
api = overpy.Overpass()

def osm_count_circle(lat, lon, radius_m, tag_key, tag_val,
                     mirrors=None, timeout=120):
    if mirrors is None:
        mirrors = [
            "https://lz4.overpass-api.de/api",
            "https://overpass.kumi.systems/api",
            "https://overpass-api.de/api",
        ]
    q = f"""
    [out:json][timeout:{timeout}];
    (
      node["{tag_key}"="{tag_val}"](around:{radius_m},{lat},{lon});
      way ["{tag_key}"="{tag_val}"](around:{radius_m},{lat},{lon});
      rel ["{tag_key}"="{tag_val}"](around:{radius_m},{lat},{lon});
    );
    out count;
    """
    for url in mirrors:
        try:
            api = overpy.Overpass(url=url, timeout=timeout)
            res = api.query(q)
            return res.count_nodes + res.count_ways + res.count_relations
        except (overpy.exception.OverpassGatewayTimeout,
                overpy.exception.OverpassTooManyRequests):
            time.sleep(10)
    raise RuntimeError("All Overpass mirrors timed out.")


In [5]:
import wbgapi as wb

def demand_drivers(country_iso):
    gdp_pc = wb.data.DataFrame("NY.GDP.PCAP.CD", country_iso, mrv=1).iloc[0,0]
    pop     = wb.data.DataFrame("SP.POP.TOTL",    country_iso, mrv=1).iloc[0,0]
    return {"gdp_pc": gdp_pc, "pop": pop}


In [6]:
growth = {"lights_delta_pct": +4.2}  # stub


In [7]:
def gap_score(category_count, drivers, growth):
    # benchmark: outlets per million people in USA
    BENCHMARK = 40     # tweak per category
    outlets_per_m = category_count / (drivers["pop"]/1_000_000)
    gap_ratio     = outlets_per_m / BENCHMARK          # < 1  means undersupply
    demand_index  = drivers["gdp_pc"] * (1 + growth.get("lights_delta_pct",0)/100)
    # scale demand so the final score is 0–100
    raw_score = (1-gap_ratio) * demand_index / 1e5
    return max(0, min(100, raw_score))


In [12]:
city   = "Santiago"
ctype  = "laundry"           # Google place type
iso3   = "CHL"

geom     = geocode_city(city, "Chile", radius_km=5)
supply   = osm_count(ctype, geom,city)
drivers  = demand_drivers(iso3)
score    = gap_score(supply, drivers, growth)

print(f"{city}: {supply} {ctype}s → Opportunity Score {score:.1f}/100")


OverpassGatewayTimeout: Server load too high

# config.py

In [13]:
# --- BigQuery ---
PROJECT   = "mindful-vial-460001-h6"
DATASET   = "boi_mvp"          # created once via infra/bigquery.sql
LOCATION  = "us-central1"

# --- Confluent ---
KAFKA_BOOTSTRAP = "pkc-zgp5j7.us-south1.gcp.confluent.cloud:9092"
KAFKA_API_KEY   = "LKH74XFTETPCQOPN"
KAFKA_SECRET    = "qUt3+K7gyZ296T2rUph0Y6fTWaDOEgzEFqWqJw6Sa1Z6ris6TfU+9gRjzEFdqi64"
TOPIC_DEMAND    = "boi.demand"
TOPIC_SUPPLY    = "boi.supply"


In [15]:
# src/ingest/worldbank.py
from confluent_kafka import Producer
from .. import config as cfg

p = Producer({
    "bootstrap.servers": confi.KAFKA_BOOTSTRAP,
    "sasl.username":     confi.KAFKA_API_KEY,
    "sasl.password":     confi.KAFKA_SECRET,
    "security.protocol": "SASL_SSL",
    "sasl.mechanisms":   "PLAIN",
})

def fetch_and_stream(iso3_list):
    ...
    for row in rows:                # rows <- list of dicts
        p.produce(confi.TOPIC_DEMAND,
                  key=row["iso3"],
                  value=json.dumps(row))
    p.flush()


ImportError: attempted relative import with no known parent package